In [1]:
from gensim.models.word2vec import Word2Vec
import pickle 
import numpy as np
import random
import string
from keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [2]:
import keras

### Load data

In [3]:
data = pd.read_csv('ner.csv', encoding = "ISO-8859-1", error_bad_lines=False)
data.head()

b'Skipping line 281837: expected 25 fields, saw 34\n'


,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [4]:
data=data.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word',"pos",'shape'],axis=1)

In [5]:
data.nunique()

sentence_idx    35177
word            30172
tag                17
dtype: int64

In [6]:
data[data.word=='police']

,sentence_idx,word,tag
657,29.0,police,O
819,35.0,police,O
886,38.0,police,O
942,41.0,police,O
969,42.0,police,O
...,...,...,...
1049492,47894.0,police,O
1049529,47896.0,police,O
1049569,47897.0,police,O
1049642,47901.0,police,O


In [7]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['word'].to_list()))
    else:
        vocab = list(set(data['tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok


token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [7]:
del idx2tag[2]

In [30]:
token2idx['unavailable']=30173

In [16]:
data['word_idx'] = data['word'].map(token2idx)
data['tag_idx'] = data['tag'].map(tag2idx) 

In [11]:
data[data['tag']=='nan']

,sentence_idx,word,tag,word_idx,tag_idx


### Transform data to sequential form

In [17]:

#data_fillna = data.fillna(method='ffill', axis=0)
#data_group = data_fillna.groupby(['sentence_idx'],as_index=False
#                                )['word',  'tag', 'word_idx', 'tag_idx'].agg(lambda x: list(x))

#data_group.head()
###

<ipython-input-17-059aaa2d111f>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(['sentence_idx'],as_index=False


,sentence_idx,word,tag,word_idx,tag_idx
0,1.0,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[6746, 3762, 28917, 24938, 9685, 2012, 589, 98...","[14, 14, 14, 14, 14, 14, 3, 14, 14, 14, 14, 14..."
1,2.0,"[Families, of, soldiers, killed, in, the, conf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[17868, 3762, 21244, 1769, 24219, 7500, 23716,...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1..."
2,3.0,"[They, marched, from, the, Houses, of, Parliam...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...","[10192, 9685, 23873, 7500, 6909, 3762, 9422, 9...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 3..."
3,4.0,"[Police, put, the, number, of, marchers, at, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2041, 20494, 7500, 651, 3762, 12523, 17510, 1...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1..."
4,5.0,"[The, protest, comes, on, the, eve, of, the, a...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,...","[12068, 22153, 7546, 56, 7500, 10003, 3762, 75...","[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 3..."


In [64]:
#data_group.to_csv('train_tag_idx.csv',index=False)

In [181]:
data_group=pd.read_csv('train_tag_idx.csv')

#### in case the dataframe dosnt work
from ast import literal_eval
data_group['word_idx']=data_group['word_idx'].apply(lambda x: literal_eval(x))
data_group['tag']=data_group['tag'].apply(lambda x: literal_eval(x))
data_group['word']=data_group['word'].apply(lambda x: literal_eval(x))
data_group['tag_idx']=data_group['tag_idx'].apply(lambda x: literal_eval(x))

In [54]:
data_group

,sentence_idx,word,tag,word_idx,tag_idx
0,1.0,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[17647, 6660, 28614, 16731, 27004, 3620, 22391...","[8, 8, 8, 8, 8, 8, 14, 8, 8, 8, 8, 8, 14, 8, 8..."
1,2.0,"[Families, of, soldiers, killed, in, the, conf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[210, 6660, 4126, 18493, 7427, 1803, 22780, 21...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
2,3.0,"[They, marched, from, the, Houses, of, Parliam...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...","[2783, 27004, 11953, 1803, 5969, 6660, 20398, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 14, 15, 8, 8..."
3,4.0,"[Police, put, the, number, of, marchers, at, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[6004, 13327, 1803, 4676, 6660, 12970, 23395, ...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
4,5.0,"[The, protest, comes, on, the, eve, of, the, a...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,...","[2064, 7783, 4679, 22309, 1803, 1167, 6660, 18...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 14, 8, 8, 6,..."
...,...,...,...,...,...
35172,47955.0,"[Indian, border, security, forces, are, accusi...","[B-gpe, O, O, O, O, O, O, B-gpe, O, O, O, O, O...","[9458, 6100, 25754, 10272, 13330, 16846, 10511...","[5, 8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 8, 8, 8, 8, ..."
35173,47956.0,"[Indian, officials, said, no, one, was, injure...","[B-gpe, O, O, O, O, O, O, O, B-tim, O, O, O, O...","[9458, 11940, 20929, 9758, 26786, 29656, 3338,...","[5, 8, 8, 8, 8, 8, 8, 8, 13, 8, 8, 8, 8, 8, 8,..."
35174,47957.0,"[Two, more, landed, in, fields, belonging, to,...","[O, O, O, O, O, O, O, O, O, O, O]","[22684, 782, 9877, 7427, 11369, 7482, 4883, 17...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
35175,47958.0,"[They, say, not, all, of, the, rockets, explod...","[O, O, O, O, O, O, O, O, O, O, O]","[2783, 2827, 28157, 7428, 6660, 1803, 25008, 2...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"


### Filter out data (for later retrain)

test=['O']
L=[]
m=[]
for i in data_group['tag']:
    for j in i:
        if all(j)=='O':
            L.append(i)
        else:
            m.append(i)

a=data_group['tag'].to_list()

len(a)

len(set(a[0]))

L=[]

for i in a:
    l=len((set(i)))
    if l>1:
        L.append(i)
    else:
        m.append(i)
        

len(L)

### download word2vec

In [ ]:
data_group=pd.read_csv('train_tag_idx.csv')

In [183]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [55]:
word2vec_vectors = gensim.downloader.load('word2vec-google-news-300')

### Generate embbedding weight for words

In [46]:
tokens=[j for i in data_group['word'].to_list() for j in i]

In [12]:
len(tokens)

1050795

In [47]:
words=list(set([i for i in tokens]))
word2idx = {w: i for i, w in enumerate(words)}

In [24]:
# pickle.dump(word2idx,open('word2idx.pickle','wb'))

In [14]:
len(word2idx)

30172

In [48]:
max_len=max([len(s)for s in data_group['word'].to_list()])
max_len

140

In [49]:
#generate random matrix for word not in word2vec corpora
vec=[]

def getitem():
    new_vec = np.array([random.random() for i in range(300)])
    vec.append(new_vec)
    return new_vec


In [50]:
embedding_matrix1=np.zeros((len(word2idx) + 2,300))

In [18]:
len(word2idx)

30172

In [51]:
def matrix(model):
    for word, v in word2idx.items():
        embedding_matrix1[-1]=getitem()
        if word in model.wv.vocab:
            embedding_matrix1[v]=model[word]
        else:
            embedding_matrix1[v]=getitem()
            
    return embedding_matrix1

In [56]:
embedding_matrix1=matrix(word2vec_vectors)

<ipython-input-51-61b0393e10ca>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.vocab:


In [21]:
len(embedding_matrix1)

30173

In [57]:
embedding_matrix1[30173]=getitem()

### pad training data

In [9]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [67]:
def get_pad_train_test_val(data_group, data):
    n_token = len(list(set(data['word'].to_list())))
    n_tag = len(list(set(data['tag'].to_list())))
    
    tokens = data_group['word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value=n_token-1)
    
    tags = data_group['tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    
    train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntrain_tags:', len(train_tags),
        '\ntest_tags:', len(test_tags)
    )
    
    return train_tokens, test_tokens, train_tags, test_tags

train_tokens, test_tokens, train_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 31659 
test_tokens length: 3518 
train_tags: 31659 
test_tags: 3518


### Train Model

In [43]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model

In [39]:
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [44]:
input_dim = len(list(set(data['word'].to_list())))+1
output_dim = 300
input_length =140
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  30174 
output_dim:  300 
input_length:  140 
n_tags:  18


In [60]:
model = Sequential()

# Add Embedding layer
model.add(Embedding(input_dim=input_dim, output_dim=output_dim,weights=[embedding_matrix1], input_length=input_length))

# Add bidirectional LSTM
model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

 # Add LSTM
model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

# Add timeDistributed Layer
model.add(TimeDistributed(Dense(n_tags, activation="softmax")))

#Optimiser 
adam = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 140, 300)          9052200   
_________________________________________________________________
bidirectional (Bidirectional (None, 140, 600)          1442400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 140, 300)          1081200   
_________________________________________________________________
time_distributed (TimeDistri (None, 140, 18)           5418      
Total params: 11,581,218
Trainable params: 11,581,218
Non-trainable params: 0
_________________________________________________________________


In [65]:
hist = model.fit(train_tokens, np.array(train_tags), batch_size=1000, verbose=1, epochs=15, validation_split=0.2)

Epoch 1/15
26/26 [==============================] - 1382s 53s/step - loss: 0.1913 - accuracy: 0.9672 - val_loss: 0.1630 - val_accuracy: 0.9676
Epoch 2/15
26/26 [==============================] - 1426s 55s/step - loss: 0.1611 - accuracy: 0.9672 - val_loss: 0.1539 - val_accuracy: 0.9676
Epoch 3/15
26/26 [==============================] - 1446s 56s/step - loss: 0.1496 - accuracy: 0.9672 - val_loss: 0.1349 - val_accuracy: 0.9676
Epoch 4/15
26/26 [==============================] - 1457s 56s/step - loss: 0.1212 - accuracy: 0.9679 - val_loss: 0.0974 - val_accuracy: 0.9718
Epoch 5/15
26/26 [==============================] - 1462s 56s/step - loss: 0.0894 - accuracy: 0.9738 - val_loss: 0.0778 - val_accuracy: 0.9765
Epoch 6/15
26/26 [==============================] - 1490s 57s/step - loss: 0.0744 - accuracy: 0.9769 - val_loss: 0.0695 - val_accuracy: 0.9779
Epoch 7/15
26/26 [==============================] - 1453s 56s/step - loss: 0.0665 - accuracy: 0.9788 - val_loss: 0.0638 - val_accuracy: 0.9800

In [66]:
model.save('bi-lstm-v3')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: bi-lstm-v3\assets


### Metrics

In [7]:
pip install seqeval

Note: you may need to restart the kernel to use updated packages.
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16176 sha256=998a1f7b4a8bfc96729fabb9115d3fc72510db73aa629d5e534a635e8e2fa6ee
  Stored in directory: c:\users\yueyu\appdata\local\pip\cache\wheels\ad\5c\ba\05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [84]:
from seqeval.metrics import classification_report,precision_score, recall_score, f1_score


In [86]:
test_pred=model.predict(test_tokens)

In [169]:
tag2idx=pickle.load(open('tag2idx_v3.pickle','rb'))
tag_ind=[]
tags=[]
for i,v in tag2idx.items():
    tags.append(i)
    tag_ind.append(v)

In [140]:
ct=[list(to_categorical(i, num_classes=18)) for i in tag_ind]
tags_nd=dict(zip(tags,ct))

In [149]:
idx2tag={}
for k,v in tag2idx.items():
    idx2tag[v]=k

In [ ]:
idx2tag[0]='O'

In [167]:
idx2tag

{0: 'O',
 1: 'I-eve',
 2: 'B-art',
 3: 'I-tim',
 4: 'I-per',
 5: 'B-gpe',
 6: 'B-org',
 7: 'I-art',
 8: 'O',
 9: 'I-org',
 10: 'I-gpe',
 11: 'B-eve',
 12: 'I-nat',
 13: 'B-tim',
 14: 'B-geo',
 15: 'I-geo',
 16: 'B-per',
 17: 'B-nat'}

In [168]:
def pred2label(pred):
    out = []
    for i in pred:
        out_i = []
        for p in i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            for k,v in tags_nd.items():
                if v==list(p):
                    out_i.append(idx2tag[k])
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = test2label(test_tags)
print(classification_report(test_labels, pred_labels))

C:\Users\yueyu\anaconda3_new\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         art       0.00      0.00      0.00        30
         eve       0.00      0.00      0.00        32
         geo       0.80      0.88      0.84      3829
         gpe       0.93      0.91      0.92      1712
         nat       0.00      0.00      0.00        21
         org       0.22      0.59      0.32      2004
         per       0.68      0.61      0.65      1698
         tim       0.83      0.82      0.82      2016

   micro avg       0.60      0.78      0.67     11342
   macro avg       0.43      0.48      0.44     11342
weighted avg       0.70      0.78      0.72     11342

